# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [2]:
from trading_bot.agent import Agent

model_name = 'GOOG1d_50'
test_stock = 'data/GOOG_1d_test.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



### 2. Load test data

In [3]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
#df = df[['Date', 'Adj Close']]
#df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})

## minute
try:
    df = df[['Datetime', 'Adj Close']] # 4 minute data
    df = df.rename(columns={'Adj Close': 'actual', 'Datetime': 'date'})
except KeyError: # date if daily, intraday Datetime
    df = df[['Date', 'Adj Close']] # 4 minute data
    df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# rename feature column names
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-30,1089.060059
1,2019-01-31,1116.369995
2,2019-02-01,1110.750000
3,2019-02-04,1132.800049
4,2019-02-05,1145.989990


### 3. Running Eval

In [4]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data["Adj Close"][1] - test_data["Adj Close"][0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG switching to TensorFlow for CPU
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $85.29
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $87.53 | Position: +$2.24
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $86.08
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $85.78
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $85.78
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $87.94 | Position: +$1.86
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $87.84 | Position: +$2.06
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $87.96 | Position: +$2.19
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $87.31
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $87.71
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $87.32
202

Data lerngth 404


2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $94.30 | Position: +$3.36
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $96.36 | Position: +$6.62
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $96.98 | Position: +$7.32
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $94.84 | Position: +$2.44
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $93.82 | Position: +$0.61
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $93.13
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $92.18
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Buy at: $91.81
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $92.20 | Position: -$0.90
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $93.94 | Position: +$0.81
2020-09-08 15:40:01 justin-desktop-ubuntu root[42729] DEBUG Sell at: $94.43 | Position: +$2.25
2020-09-08 15:4

2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $95.93 | Position: +$5.94
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $98.21 | Position: +$9.55
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $97.79 | Position: +$8.50
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $96.45
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $95.95
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $94.81
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $92.47
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $93.39
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $95.13
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $95.02 | Position: +$6.20
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $93.50
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $94.58 | Position: +$0.68


2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $100.62
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $104.29 | Position: -$5.57
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $104.80
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $106.79
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $106.48 | Position: -$3.77
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $108.55 | Position: +$4.47
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $109.85 | Position: +$4.02
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell at: $111.07 | Position: +$1.16
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $108.81
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $106.65
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Buy at: $107.21
2020-09-08 15:40:02 justin-desktop-ubuntu root[42729] DEBUG Sell

Final decision:  BUY  at  $130.65


In [5]:
#+$294.03

### 4. Visualize

In [6]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 0.1, max(max(df['actual']), max(df['position'])) + 0.1), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [7]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)